In [0]:
!pip install dgl

     |████████████████████████████████| 3.0MB 2.8MB/s 


In [0]:
#Manipulating heterogeneous graph
import dgl
import scipy.io
import urllib.request

data_url = 'https://data.dgl.ai/dataset/ACM.mat'
data_file_path = '/tmp/ACM.mat'

urllib.request.urlretrieve(data_url, data_file_path)
data = scipy.io.loadmat(data_file_path)
print(list(data.keys()))

['__header__', '__version__', '__globals__', 'TvsP', 'PvsA', 'PvsV', 'AvsF', 'VvsC', 'PvsL', 'PvsC', 'A', 'C', 'F', 'L', 'P', 'T', 'V', 'PvsT', 'CNormPvsA', 'RNormPvsA', 'CNormPvsC', 'RNormPvsC', 'CNormPvsT', 'RNormPvsT', 'CNormPvsV', 'RNormPvsV', 'CNormVvsC', 'RNormVvsC', 'CNormAvsF', 'RNormAvsF', 'CNormPvsL', 'RNormPvsL', 'stopwords', 'nPvsT', 'nT', 'CNormnPvsT', 'RNormnPvsT', 'nnPvsT', 'nnT', 'CNormnnPvsT', 'RNormnnPvsT', 'PvsP', 'CNormPvsP', 'RNormPvsP']


In [0]:
print(type(data['PvsA']))
print('#Papers:', data['PvsA'].shape[0])
print('#Authors:', data['PvsA'].shape[1])
print('#Links:', data['PvsA'].nnz)

<class 'scipy.sparse.csc.csc_matrix'>
#Papers: 12499
#Authors: 17431
#Links: 37055


In [0]:
#scipy matrix to heterogeneous graph
pa_g = dgl.heterograph({('paper', 'written-by', 'author') : data['PvsA']})
# equivalent (shorter) API for creating heterograph with two node types:
pa_g = dgl.bipartite(data['PvsA'], 'paper', 'written-by', 'author')

In [0]:
print('Node types:', pa_g.ntypes)
print('Edge types:', pa_g.etypes)
print('Canonical edge types:', pa_g.canonical_etypes)

Node types: ['paper', 'author']
Edge types: ['written-by']
Canonical edge types: [('paper', 'written-by', 'author')]


In [0]:
print(pa_g.number_of_nodes('paper'))

12499


In [0]:
print(pa_g.number_of_edges(('paper', 'written-by', 'author')))

37055


In [0]:
# Paper-citing-paper graph is a homogeneous graph
pp_g = dgl.heterograph({('paper', 'citing', 'paper') : data['PvsP']})

In [0]:
# equivalent (shorter) API for creating homogeneous graph
pp_g = dgl.graph(data['PvsP'], 'paper', 'cite')

In [0]:
# All the ntype and etype arguments could be omitted because the behavior is unambiguous.
print(pp_g.number_of_nodes())
print(pp_g.number_of_edges())
print(pp_g.successors(3))

12499
30789
tensor([1361, 2624, 8670, 9845])


In [0]:
G = dgl.heterograph({
        ('paper', 'written-by', 'author') : data['PvsA'],
        ('author', 'writing', 'paper') : data['PvsA'].transpose(),
        ('paper', 'citing', 'paper') : data['PvsP'],
        ('paper', 'cited', 'paper') : data['PvsP'].transpose(),
        ('paper', 'is-about', 'subject') : data['PvsL'],
        ('subject', 'has', 'paper') : data['PvsL'].transpose(),
        ('paper', 'published-in', 'conference') : data['PvsC']
    })

print(G)

Graph(num_nodes={'author': 17431, 'conference': 14, 'paper': 12499, 'subject': 73},
      num_edges={('paper', 'written-by', 'author'): 37055, ('author', 'writing', 'paper'): 37055, ('paper', 'citing', 'paper'): 30789, ('paper', 'cited', 'paper'): 30789, ('paper', 'is-about', 'subject'): 12499, ('subject', 'has', 'paper'): 12499, ('paper', 'published-in', 'conference'): 12499},
      metagraph=[('author', 'paper'), ('paper', 'author'), ('paper', 'paper'), ('paper', 'paper'), ('paper', 'subject'), ('paper', 'conference'), ('subject', 'paper')])


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

pvc = data['PvsC'].tocsr()
# find all papers published in KDD, ICML, VLDB
c_selected = [0, 11, 13]  # KDD, ICML, VLDB
p_selected = pvc[:, c_selected].tocoo()
# generate labels
labels = pvc.indices
labels[labels == 11] = 1
labels[labels == 13] = 2
labels = torch.tensor(labels).long()

# generate train/val/test split
pid = p_selected.row
shuffle = np.random.permutation(pid)
train_idx = torch.tensor(shuffle[0:800]).long()
val_idx = torch.tensor(shuffle[800:900]).long()
test_idx = torch.tensor(shuffle[900:]).long()

In [0]:
G.ntypes

['author', 'conference', 'paper', 'subject']

In [0]:
G.number_of_nodes('author')

17431

In [0]:
nn.Parameter(torch.Tensor(G.number_of_nodes('author'), 10))

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)

In [0]:
import dgl.function as fn

class HeteroRGCNLayer(nn.Module):
    def __init__(self, in_size, out_size, etypes):
        super(HeteroRGCNLayer, self).__init__()
        # W_r for each relation
        #seperate weight relations are formed for each of the edge types-'written-by', 'writing', 'citing', 'cited', 'is-about', 'has', 'published-in'
        self.weight = nn.ModuleDict({
                name : nn.Linear(in_size, out_size) for name in etypes
            })

    def forward(self, G, feat_dict):
        # The input is a dictionary of node features for each type
        funcs = {}
        for srctype, etype, dsttype in G.canonical_etypes:
            # Compute W_r * h
            #self.weight['written-by'](feat_dict['paper'])
            Wh = self.weight[etype](feat_dict[srctype])
            # Save it in graph for message passing
            #G.nodes['subject'].data['Wh_has']
            G.nodes[srctype].data['Wh_%s' % etype] = Wh
            # Specify per-relation message passing functions: (message_func, reduce_func).
            # Note that the results are saved to the same destination feature 'h', which
            # hints the type wise reducer for aggregation.
            funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
        # Trigger message passing of multiple types.
        # The first argument is the message passing functions for each relation.
        # The second one is the type wise reducer, could be "sum", "max",
        # "min", "mean", "stack"
        G.multi_update_all(funcs, 'sum')
        # return the updated node feature dictionary
        return {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}

In [0]:
G.canonical_etypes

[('paper', 'written-by', 'author'),
 ('author', 'writing', 'paper'),
 ('paper', 'citing', 'paper'),
 ('paper', 'cited', 'paper'),
 ('paper', 'is-about', 'subject'),
 ('subject', 'has', 'paper'),
 ('paper', 'published-in', 'conference')]

In [0]:
HeteroRGCNLayer(10,10,G.etypes)

HeteroRGCNLayer(
  (weight): ModuleDict(
    (cited): Linear(in_features=10, out_features=10, bias=True)
    (citing): Linear(in_features=10, out_features=10, bias=True)
    (has): Linear(in_features=10, out_features=10, bias=True)
    (is-about): Linear(in_features=10, out_features=10, bias=True)
    (published-in): Linear(in_features=10, out_features=10, bias=True)
    (writing): Linear(in_features=10, out_features=10, bias=True)
    (written-by): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [0]:
class HeteroRGCN(nn.Module):
    def __init__(self, G, in_size, hidden_size, out_size):
        super(HeteroRGCN, self).__init__()
        # Use trainable node embeddings as featureless inputs.
        embed_dict = {ntype : nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), in_size))
                      for ntype in G.ntypes}
        for key, embed in embed_dict.items():
            nn.init.xavier_uniform_(embed)
        self.embed = nn.ParameterDict(embed_dict)
        # create layers
        self.layer1 = HeteroRGCNLayer(in_size, hidden_size, G.etypes)
        self.layer2 = HeteroRGCNLayer(hidden_size, out_size, G.etypes)

    def forward(self, G):
        h_dict = self.layer1(G, self.embed)
        h_dict = {k : F.leaky_relu(h) for k, h in h_dict.items()}
        h_dict = self.layer2(G, h_dict)
        # get paper logits
        return h_dict['paper']

In [0]:
G.ntypes

['author', 'conference', 'paper', 'subject']

In [0]:
# Create the model. The output has three logits for three classes.
model = HeteroRGCN(G, 10, 10, 3)

In [0]:
# Create the model. The output has three logits for three classes.
model = HeteroRGCN(G, 10, 10, 3)

opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

best_val_acc = 0
best_test_acc = 0

for epoch in range(100):
    logits = model(G)
    # The loss is computed only for labeled nodes.
    loss = F.cross_entropy(logits[train_idx], labels[train_idx])

    pred = logits.argmax(1)
    train_acc = (pred[train_idx] == labels[train_idx]).float().mean()
    val_acc = (pred[val_idx] == labels[val_idx]).float().mean()
    test_acc = (pred[test_idx] == labels[test_idx]).float().mean()

    if best_val_acc < val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc

    opt.zero_grad()
    loss.backward()
    opt.step()

    if epoch % 5 == 0:
        print('Loss %.4f, Train Acc %.4f, Val Acc %.4f (Best %.4f), Test Acc %.4f (Best %.4f)' % (
            loss.item(),
            train_acc.item(),
            val_acc.item(),
            best_val_acc.item(),
            test_acc.item(),
            best_test_acc.item(),
        ))

Loss 1.4980, Train Acc 0.3688, Val Acc 0.3300 (Best 0.3300), Test Acc 0.3467 (Best 0.3467)
Loss 1.1178, Train Acc 0.4137, Val Acc 0.3400 (Best 0.3400), Test Acc 0.3610 (Best 0.3610)
Loss 0.8335, Train Acc 0.6587, Val Acc 0.4600 (Best 0.6800), Test Acc 0.5603 (Best 0.7085)
Loss 0.6791, Train Acc 0.6400, Val Acc 0.4500 (Best 0.6800), Test Acc 0.5528 (Best 0.7085)
Loss 0.5032, Train Acc 0.8338, Val Acc 0.5700 (Best 0.6800), Test Acc 0.6943 (Best 0.7085)
Loss 0.3598, Train Acc 0.9187, Val Acc 0.6600 (Best 0.6800), Test Acc 0.7571 (Best 0.7085)
Loss 0.2267, Train Acc 0.9625, Val Acc 0.7400 (Best 0.7400), Test Acc 0.8007 (Best 0.8007)
Loss 0.1329, Train Acc 0.9937, Val Acc 0.7500 (Best 0.7500), Test Acc 0.8074 (Best 0.8132)
Loss 0.0823, Train Acc 0.9975, Val Acc 0.7500 (Best 0.7500), Test Acc 0.7973 (Best 0.8132)
Loss 0.0559, Train Acc 1.0000, Val Acc 0.7500 (Best 0.7500), Test Acc 0.7923 (Best 0.8132)
Loss 0.0420, Train Acc 1.0000, Val Acc 0.7500 (Best 0.7500), Test Acc 0.7889 (Best 0.8132)